In [15]:
from time import sleep
import pyautogui
from os import path
from random import uniform, seed
import math
import pprint
from copy import deepcopy

In [2]:
seed()

In [3]:
ANIM_SECONDS = 0.01
CUR_DIR = "C:\\Users\\Shevis\\src\\github.com\\shevisj\\splitbuilder"
BUILD_MODE = "move_speed.png"
OBJECTS_MENU = "objects_menu.png"
TOOLS_MENU = "tools_menu.png"
MOVEMENT_MENU = "movement_menu.png"
ROTATION_MENU = "rotation_menu.png"
BLOCKS_MENU = "blocks_menu.png"
RAMPS_MENU = "blocks_ramps_menu.png"
FLOORS_MENU = "blocks_floors_menu.png"
GLASS_RAMP = "glass_ramp.png"
GLASS_FLOOR = "glass_floor.png"
ENVIRONMENT_MENU = "environment_menu.png"

In [4]:
X_POS = (216, 935)
Y_POS = (216, 1107)
Z_POS = (216, 1280)
SURF_RAMP_ANGLE = -31
DOWN_STRAIGHT_START_ROT = (-135, 0, -45)

# Types

In [12]:
class Vector3:
    def __init__(self, x=0, y=0, z=0):
        self.x = x
        self.y = y
        self.z = z
    
    def __repr__(self):
        return f"(x: {self.x}, y: {self.y}, z: {self.z})"

class Transform:
    def __init__(self, position: Vector3, rotation: Vector3):
        self.position = position
        self.rotation = rotation
    
    def forward(self, distance: int) -> Vector3:
        pos_x = self.position.x - round(math.sin(-1.0 * math.radians(self.rotation.y)) * distance)
        pos_z = self.position.z - round(math.cos(-1.0 * math.radians(self.rotation.y)) * distance)
        return Vector3(pos_x, self.position.y, pos_z)
    
    def copy(self, position: Vector3 = None, rotation: Vector3 = None):
        new = deepcopy(self)
        if position is not None:
            new.position = position
        if rotation is not None:
            new.rotation = rotation
        return new

    def __repr__(self):
        return f"Transform(\n\tposition: {self.position},\n\trotation: {self.rotation},\n)"


# Start Game

In [19]:
def find_icon(icon_name, no_print=False):
    # Find the game icon
    image_path = path.join(CUR_DIR, 'images', icon_name)
    try:
        game_icon = pyautogui.locateOnScreen(image_path, confidence=0.8)
        if game_icon:
            if not no_print:
                print(f"{icon_name} found: {game_icon}")
            return pyautogui.center(game_icon)
        return None
    except pyautogui.ImageNotFoundException:
        if not no_print:
            print("Unable to locate image file")
        return None

def click_icon(icon_name, no_print=False):
    game_icon = find_icon(icon_name=icon_name, no_print=no_print)
    if game_icon:
        # Start the game
        pyautogui.moveTo(game_icon.x, game_icon.y, duration=ANIM_SECONDS)
        pyautogui.click(game_icon)
    elif not no_print:
        print("Unable to locate game icon", icon_name)

def click_spot(x, y, dur=ANIM_SECONDS):
    pyautogui.moveTo(x, y)
    pyautogui.click(x=x, y=y)

# Movement

In [7]:
def walk_wasd(dur=2):
    pyautogui.keyDown('w')
    sleep(dur)
    pyautogui.keyUp('w')

# Building

In [23]:
def is_active(img):
    icon_path = path.join(CUR_DIR, 'images', img)
    icon = pyautogui.locateOnScreen(icon_path, confidence=0.8)
    return icon is not None

def activate_build_mode():
    if not is_active(BUILD_MODE):
        pyautogui.press('`')

def open_build_menu():
    if not is_active(OBJECTS_MENU):
        pyautogui.press('m')

def reset_build_menu():
    open_build_menu()
    click_icon(OBJECTS_MENU)
    for _ in range(3):
        if not is_active(ENVIRONMENT_MENU):
            pyautogui.press('escape')
            continue
        break

def enter_value(spot, val):
    click_spot(spot[0], spot[1])
    val_chars = list(str(val))
    for ch in val_chars:
        pyautogui.press(ch)
    pyautogui.press('enter')

def place_object(nav, transform: Transform):
    click_icon(OBJECTS_MENU)
    for menu in nav:
        click_icon(menu, no_print=True)
    click_icon(TOOLS_MENU)
    click_icon(MOVEMENT_MENU)
    enter_value(X_POS, transform.position.x)
    enter_value(Y_POS, transform.position.y)
    enter_value(Z_POS, transform.position.z)
    click_icon(ROTATION_MENU)
    enter_value(X_POS, transform.rotation.x)
    enter_value(Y_POS, transform.rotation.y)
    enter_value(Z_POS, transform.rotation.z)
    click_spot(1122, 1122)

def straight_panal_extrapolator(transform: Transform, n_panals):
    pos_x = transform.position.x
    pos_y = transform.position.y
    pos_z = transform.position.z
    rot_x = transform.rotation.x
    rot_y = transform.rotation.y
    rot_z = transform.rotation.z
    panal_width = 400
    sign = math.copysign(1, n_panals)
    panal_mag = abs(n_panals)
    def extrapolate():
        nonlocal pos_x, pos_z
        for _ in range(panal_mag - 1):
            pos_x = pos_x + round(math.sin(-1.0 * math.radians(rot_y)) * panal_width * sign)
            pos_z = pos_z + round(math.cos(-1.0 * math.radians(rot_y)) * panal_width * sign)
            yield Transform(
                position=Vector3(pos_x, pos_y, pos_z),
                rotation=Vector3(rot_x, rot_y, rot_z)
            )
    return extrapolate

def place_surf_straight(transform: Transform, n_panals):
    reset_build_menu()
    place_object(
        [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP],
        transform
    )
    spe = straight_panal_extrapolator(transform, n_panals=n_panals)
    for new_transform in spe():
        place_object(
            [GLASS_RAMP],
            new_transform
        )

def calc_sep_deg(radius):
    return math.degrees(math.atan(400.0 / (float(radius) + (800 * math.cos(math.radians(45))))))

# Radii is a list of 2-tuples with radius, count pairs
def flex_curv_extrapolator(transform: Transform, radii, sign=1, h_deltas=None):
    pos_x = transform.position.x
    pos_y = transform.position.y
    pos_z = transform.position.z
    rot_x = transform.rotation.x
    rot_y = transform.rotation.y
    rot_z = transform.rotation.z
    n_panals = sum([r[1] for r in radii])
    if h_deltas is not None:
        assert (isinstance(h_deltas, list))
        n_deltas = sum([r[1] for r in h_deltas])
        assert n_deltas == n_panals
        result = []
        for delta in h_deltas:
            result.extend([delta[0]] * delta[1])
        h_deltas = result
    else:
        h_deltas = [0] * n_panals
    def extrapolate():
        cur_pos_x = pos_x
        cur_pos_z = pos_z
        cur_rot_y = rot_y
        cur_pos_y = pos_y
        i = -1
        for cur_radius in radii:
            cur_center_x = cur_pos_x + round(math.sin(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0] * -1.0)
            cur_center_z = cur_pos_z + round(math.cos(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0] * -1.0)
            deg_offset = calc_sep_deg(cur_radius[0]) * sign
            for _ in range(cur_radius[1]):
                i += 1
                cur_pos_y = cur_pos_y + h_deltas[i]
                cur_rot_y = mod_rot(round(cur_rot_y + deg_offset))
                cur_pos_x = cur_center_x + round(math.sin(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0])
                cur_pos_z = cur_center_z + round(math.cos(-1.0 * math.radians(cur_rot_y + 90)) * cur_radius[0])
                yield Transform(
                    position=Vector3(cur_pos_x, cur_pos_y, cur_pos_z),
                    rotation=Vector3(rot_x, cur_rot_y, rot_z)
                )
    return extrapolate

def place_flex_curv(transform: Transform, radii, sign=1, h_deltas=None, ignore_first=False):
    reset_build_menu()
    if not ignore_first:
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP],
            transform
        )
    fce = flex_curv_extrapolator(transform, radii, sign, h_deltas)
    first = True
    for new_transform in fce():
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP] if ignore_first and first else [GLASS_RAMP],
            new_transform
        )
        first = False

def mod_rot(rot):
    currot = rot
    while currot < -180:
        currot += 360
    return ((rot + 180) % 360) - 180
        

def curv_extrapolator(transform: Transform, radius, n_panals=5, h_delta=0, r_delta=0):
    pos_x = transform.position.x
    pos_y = transform.position.y
    pos_z = transform.position.z
    rot_x = transform.rotation.x
    rot_y = transform.rotation.y
    rot_z = transform.rotation.z
    abs_radius = abs(radius)
    sign = math.copysign(1, n_panals)
    abs_n_panals = abs(n_panals)
    center_x = pos_x + round(math.sin(-1.0 * math.radians(rot_y + 90)) * abs_radius * -1.0)
    center_z = pos_z + round(math.cos(-1.0 * math.radians(rot_y + 90)) * abs_radius * -1.0)
    def extrapolate():
        deg_offset = 0
        cur_pos_y = pos_y
        cur_abs_radius = abs_radius
        for _ in range(abs_n_panals - 1):
            
            cur_pos_y += h_delta
            cur_abs_radius += r_delta

            deg_offset += calc_sep_deg(cur_abs_radius) * sign
            new_x = center_x + round(math.sin(-1.0 * math.radians(rot_y + 90 + deg_offset)) * cur_abs_radius)
            new_z = center_z + round(math.cos(-1.0 * math.radians(rot_y + 90 + deg_offset)) * cur_abs_radius)
            yield Transform(
                position=Vector3(new_x, cur_pos_y, new_z),
                rotation=Vector3(rot_x, mod_rot(round(rot_y + deg_offset)), rot_z)
            )
    return extrapolate

def place_curv(transform: Transform, radius: int, n_panals=5, h_delta=0, r_delta=0, ignore_first=False):
    reset_build_menu()
    if not ignore_first:
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP],
            transform
        )
    ce = curv_extrapolator(transform, radius, n_panals, h_delta, r_delta)
    first = True
    for new_transform in ce():
        place_object(
            [OBJECTS_MENU, BLOCKS_MENU, RAMPS_MENU, GLASS_RAMP] if ignore_first and first else [GLASS_RAMP],
            new_transform
        )
        first = False
    

# Composite structures

In [30]:
roots = [
    Transform(
        position=Vector3(0, 19000, 19800),
        rotation=Vector3(0, 0, 0)
    ),
    Transform(
        position=Vector3(11638, 19000, 16019),
        rotation=Vector3(0, 36, 0)
    ),
    Transform(
        position=Vector3(18831, 19000, 6119),
        rotation=Vector3(0, 72, 0)
    ),
    Transform(
        position=Vector3(18831, 19000, -6119),
        rotation=Vector3(0, 108, 0)
    ),
    Transform(
        position=Vector3(11638, 19000, -16019),
        rotation=Vector3(0, 144, 0)
    ),
    Transform(
        position=Vector3(0, 19000, -19800),
        rotation=Vector3(0, 180, 0)
    ),
    Transform(
        position=Vector3(-11638, 19000, -16019),
        rotation=Vector3(0, -144, 0)
    ),
    Transform(
        position=Vector3(-18831, 19000, -6119),
        rotation=Vector3(0, -108, 0)
    ),
    Transform(
        position=Vector3(-188310, 19000, 6119),
        rotation=Vector3(0, -72, 0)
    ),
    Transform(
        position=Vector3(-11638, 19000, 16019),
        rotation=Vector3(0, -36, 0)
    ),
]

def build_whirlwind_arm(root: Transform):
    reset_build_menu()
    # Place root
    place_object(
        [BLOCKS_MENU, FLOORS_MENU, GLASS_FLOOR],
        root
    )

    # Section A
    section_a_start_pos = root.forward(2000)
    section_a_start_pos.y -= 1000

    place_curv(
        Transform(
            position=section_a_start_pos,
            rotation=Vector3(root.rotation.x, root.rotation.y, -31)
        ),
        10000,
        20,
        -200,
        -200
    )

    #place_flex_curv(
    #    Transform(
    #        position=section_a_start_pos,
    #        rotation=Vector3(root.rotation.x, root.rotation.y, -31)
    #    ),
    #   [
    #        (5000, 1),
    #        (4000, 2),
    #        (3000, 3),
    #        (4000, 2),
    #        (5000, 1),
    #    ],
    #    1,
    #    [
    #        (-50, 9)
    #    ]
    #)
    

# Runner

In [32]:
try:
    click_icon('game_icon.png')
    activate_build_mode()
    open_build_menu()
    build_whirlwind_arm(roots[0])
    #place_flex_curv(
    #    Transform(
    #        position=Vector3(12646, 17890, -14235),
    #        rotation=Vector3(0, -45, -31)
    #    ),
    #    [
    #        (5000, 1),
    #        (4000, 2),
    #        (3000, 3),
    #        (4000, 2),
    #        (5000, 1),
    #    ],
    #    -1,
    #    [
    #        (-10, 1),
    #        (-15, 2),
    #        (-20, 3),
    #        (-25, 2),
    #        (-30, 1),
    #    ]
    #)
    #place_curv(
    #    (2646, 18890, -14235),
    #    (0, 100, -31),
    #    1500,
    #    28,
    #    -50,
    #    -50
    #)
except KeyboardInterrupt as e:
    print("Exiting")

game_icon.png found: Box(left=1321, top=2115, width=30, height=20)
objects_menu.png found: Box(left=315, top=371, width=144, height=24)
objects_menu.png found: Box(left=315, top=371, width=144, height=24)
tools_menu.png found: Box(left=557, top=371, width=104, height=24)
movement_menu.png found: Box(left=401, top=572, width=142, height=20)
rotation_menu.png found: Box(left=628, top=572, width=125, height=20)
objects_menu.png found: Box(left=315, top=371, width=144, height=24)
objects_menu.png found: Box(left=315, top=371, width=144, height=24)
tools_menu.png found: Box(left=557, top=371, width=104, height=24)
movement_menu.png found: Box(left=401, top=572, width=142, height=20)
rotation_menu.png found: Box(left=628, top=572, width=125, height=20)
objects_menu.png found: Box(left=315, top=371, width=144, height=24)
tools_menu.png found: Box(left=557, top=371, width=104, height=24)
movement_menu.png found: Box(left=401, top=572, width=142, height=20)
rotation_menu.png found: Box(left=628

In [25]:
image = pyautogui.screenshot()
image.save('testing.png')

In [ ]:
ep = straight_panal_extrapolator((0, 0, 0), (0, 0, 0), 4)

In [ ]:
for p, r in ep():
    print(p, r)

(400, 0, 0) (0, 0, 0)
(800, 0, 0) (0, 0, 0)
(1200, 0, 0) (0, 0, 0)


In [ ]:
curv_extrapolator((0, 5000, 0), (0, -90, -31), 1000, 5)

0 5000 -1000


In [ ]:
ce = curv_extrapolator((-1546, 12890, 1235), (0, 0, -31), 1000, 5)

In [ ]:
print((-1546, 12890, 1235), (0, 0, -31))
for loc, rot in ce():
    print(loc, rot)

(-1546, 12890, 1235) (0, 0, -31)
(-1542, 12890, 1148) (0, 5.0, -31)
(-1531, 12890, 1061) (0, 10.0, -31)
(-1512, 12890, 976) (0, 15.0, -31)
(-1486, 12890, 893) (0, 20.0, -31)


In [ ]:
math.degrees(math.atan(400.0 / float(2000)))

11.309932474020215

In [ ]:
800 * math.cos(math.radians(31))

685.7338405616898

In [ ]:
math.cos(math.radians(31))

0.8571673007021123

In [74]:
mod_rot(-66666)

-66

In [85]:
fce = flex_curv_extrapolator((-1546, 12890, 1235), (0, 0, -31), [(1000, 2), (500, 3), (1000, 2)])

In [87]:
print((-1546, 12890, 1235), (0, 0, -31))
for loc, rot in fce():
    print(loc, rot)

(-1546, 12890, 1235) (0, 0, -31)
(-1516, 12890, 993) (0, 14, -31)
(-1277, 12890, 553) (0, 43, -31)
(-1130, 12890, 445) (0, 64, -31)
(-782, 12890, 411) (0, 105, -31)
(-424, 12890, 782) (0, 167, -31)
(-398, 12890, 1024) (0, -179, -31)
(-532, 12890, 1507) (0, -150, -31)


In [21]:
for i in range(-180, 180, 36):
    print(i)

-180
-144
-108
-72
-36
0
36
72
108
144


In [39]:
r = roots[0]

In [40]:
r

Transform(
	position: (x: 0, y: 1900, z: 19800),
	rotation: (x: 0, y: 0, z: 0),
)

In [41]:
r.forward(1000)

(x: 0, y: 1900, z: 20800)